In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import plotnine as p9


In [ ]:
X_train = pd.read_csv("./data/external/train_logs.csv")

X_train = (
    X_train
    .sort_values(["id", "event_id"], ascending=[True, True])
    )

In [ ]:
PUNCTUATION = X_train.loc[(
    (X_train['activity'] == 'Input')
    & (~ X_train['text_change'].isin(['q', ' ']))
    ), 'text_change'].unique()

In [ ]:
# no explicit record for a pause. pauses are omitted.
PAUSE_THRESHOLD_MS = 1000

X_train['up_time_lag1'] = (
    X_train
    .groupby(['id'])
    ['up_time']
    .shift(1)
    )
# latency does not mean a meaningful pause
X_train['latency_time'] = (
    X_train['down_time'] - X_train['up_time_lag1']
    )
X_train['preceding_pause_time'] = X_train['latency_time']
# expect some negative pause times -- interpret as, no real pause
has_no_real_pause = X_train['preceding_pause_time'] <= PAUSE_THRESHOLD_MS
X_train.loc[has_no_real_pause, 'preceding_pause_time'] = None
# not obvious how to tag "initial planning pause" 
X_train['preceding_pause_time_start_window'] = X_train['preceding_pause_time']
X_train.loc[X_train['up_time'] > 5 * 60 * 1000, 'preceding_pause_time_start_window'] = None

X_train['total_pause_time'] = (
    X_train
    .groupby(['id'])
    ['preceding_pause_time']
    .transform('sum')
    )
X_train['rolling_pause_time'] = (
    X_train
    .groupby(['id'])
    ['preceding_pause_time']
    .cumsum()
    )
X_train['rolling_pause_time_fraction'] = (
    X_train['rolling_pause_time'] / X_train['total_pause_time']
    )

# summarize pause distr
MS_IN_PAUSE_BUCKET_MAX = 200e3
PAUSE_BUCKET_STEP_MS = 500

X_train['preceding_pause_time_bucket'] = pd.cut(
    X_train['preceding_pause_time'],
    bins=np.arange(
        0, 
        MS_IN_PAUSE_BUCKET_MAX,
        PAUSE_BUCKET_STEP_MS
        )
    )

X_train['preceding_pause_time_bucket'].value_counts()

# WARNING: this representation of pause distribution is dense & large
# a few parameters from distribution model far more succinct

In [ ]:
# if pause exceeds threshold duration, a "burst" has ended
MS_PER_S = 1000
SECONDS_PER_BURST = 2

X_train['is_new_burst_start'] = (
    X_train['preceding_pause_time'] > MS_PER_S * SECONDS_PER_BURST
    ).astype(int)
X_train['is_new_burst_start'][0] = 1
X_train['burst_id'] = (
    X_train
    .groupby(['id'])
    ['is_new_burst_start']
    .cumsum()
    )
X_train['burst_time_start'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['down_time']
    .transform('min')
    )
X_train['burst_time_end'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['up_time']
    .transform('max')
    )
X_train['burst_duration'] = X_train['burst_time_end'] - X_train['burst_time_start']

In [ ]:
# one-way cursor movement might be most productive
# jumping around is choppy
X_train['cursor_position_lag1'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .shift(1)
    )

X_train['has_cursor_position_moved_right'] = (
    X_train['cursor_position'] > X_train['cursor_position_lag1']
    ).astype(int)

# farthest position cursor has _edited_, with recorded input
X_train['cursor_position_cummax'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .cummax()
    )
X_train.loc[X_train['activity'] != 'Input', 'cursor_position_cummax'] = None
X_train['cursor_position_cummax'] = (
    X_train
    .groupby(['id'])
    ['cursor_position_cummax']
    .ffill()
    )

X_train['cursor_position_vs_max'] = (
    X_train['cursor_position'] - X_train['cursor_position_cummax']
    )

In [ ]:
# word count offers a productivity measure
X_train['word_count_lag1'] = (
    X_train
    .groupby(['id'])
    ['word_count']
    .shift(1)
    )

X_train['word_count_delta_event'] = (
    X_train['word_count'] - X_train['word_count_lag1']
    )

X_train['word_count_delta_burst'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['word_count_delta_event']
    .transform('sum')
    )

# word length offers a content quality measure.
# hard to track entire words sequence in rolling fashion.
    # every word's length, in a list of one element per word?  
# more tractable to track very latest string

is_edit_to_latest_string = X_train['cursor_position_vs_max'] == 0

X_train['is_new_latest_string_start'] = (
    is_edit_to_latest_string
    & (X_train['activity'] == "Input")
    & (X_train["text_change"] == ' ')
    )

X_train['is_latest_string_end'] = (
    X_train
    .groupby(['id'])
    ['is_new_latest_string_start']
    .shift(-1)
    # last process records
    .fillna(True)
    )

X_train['n_alphanum_char_added_to_latest_string'] = 0
is_alphanumeric_addition = (
    (X_train['activity'] == "Input")
    & (X_train["text_change"] == 'q')
    )
X_train.loc[
    (is_alphanumeric_addition & is_edit_to_latest_string), 
    'n_alphanum_char_added_to_latest_string'
    ] = 1
is_alphanumeric_subtraction = (
    (X_train['activity'] == "Remove/Cut")
    & (X_train['up_event'] == 'Backspace')
    & (X_train["text_change"] == 'q')
    )
X_train.loc[
    (is_alphanumeric_subtraction & is_edit_to_latest_string), 
    'n_alphanum_char_added_to_latest_string'
    ] = -1
# example: 2nd string, 2 characters in.
# considering cumsum for each character in 2nd string, 
# subtract those characters from 1st
X_train['rolling_length_latest_string'] = (
    X_train
    .groupby(['id'])
    ['n_alphanum_char_added_to_latest_string']
    .cumsum() 
    ) - (
    X_train
    .groupby(['id'])
    ['n_alphanum_char_added_to_latest_string']
    .cumsum()
    .where(X_train['is_new_latest_string_start']) 
    .ffill()
    .fillna(0)
    )

X_train['length_latest_string'] = None
X_train.loc[X_train['is_latest_string_end'], 'length_latest_string'] = X_train['rolling_length_latest_string']

In [ ]:
X_train.head(20)

In [ ]:
# if thoughts aren't separated by punctuation, writing won't score well
X_train['is_thought_delimiting_punctuation'] = (
    (X_train['text_change'] == ".")
    | (X_train['text_change'] == ". ")
    | (X_train['text_change'] == ",")
    | (X_train['text_change'] == "-")
    | (X_train['text_change'] == "!")
    | (X_train['text_change'] == ";")
    | (X_train['text_change'] == "?")
    | (X_train['text_change'] == ":")
    ).astype(int)

X_train['is_special_punctuation'] = (
    (X_train['text_change'] == "=")
    | (X_train['text_change'] == "/")
    | (X_train['text_change'] == "\\")
    | (X_train['text_change'] == "(")
    | (X_train['text_change'] == ")")
    | (X_train['text_change'] == "\n")
    | (X_train['text_change'] == "[")
    | (X_train['text_change'] == "]")
    | (X_train['text_change'] == ">")
    | (X_train['text_change'] == "<")
    | (X_train['text_change'] == "$")
    | (X_train['text_change'] == "*")
    | (X_train['text_change'] == "&")
)

In [ ]:
# windows allow for time-sequence features
TOTAL_MIN = 30
SECONDS_PER_MIN = 60
SECONDS_PER_WINDOW = 30

X_train['window_30s'] = pd.cut(
    X_train['down_time'],
    bins=np.arange(
        0, 
        TOTAL_MIN * SECONDS_PER_MIN * MS_PER_S + 5*MS_PER_S*2, 
        SECONDS_PER_WINDOW * MS_PER_S
        )
    )

In [ ]:
ACTIVITY_CATEGORIES = ['Nonproduction', 'Input', 'Remove/Cut', 'Replace', 'Paste']

pipeline_activity_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ ACTIVITY_CATEGORIES ], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["activity"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_activity_onehot.fit(X_train)
original_categorical = X_train['activity']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_activity_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_activity_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

In [ ]:
for activity in ACTIVITY_CATEGORIES:

    X_train['burst_action_time_' + activity] = (
        X_train
        .assign(activity_x_event_time = lambda x: x['activity_' + activity] * x.action_time)
        .groupby(['id', 'burst_id'])
        ['activity_x_event_time']
        .transform('sum')
        ).astype(float)
    
X_train['burst_type'] = (
    X_train
    [['burst_action_time_' + activity for activity in ACTIVITY_CATEGORIES]]
    .idxmax(axis=1)
    )
X_train['burst_type'] = (
    X_train['burst_type']
    .str
    .replace("burst_action_time_", "", regex=True)
    )

In [ ]:
pipeline_burst_type_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ ACTIVITY_CATEGORIES ], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["burst_type"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_burst_type_onehot.fit(X_train)
original_categorical = X_train['burst_type']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_burst_type_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_burst_type_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

In [ ]:
for activity in ACTIVITY_CATEGORIES:

    X_train['is_new_burst_start_' + activity] = (
        X_train['is_new_burst_start'] * 
        X_train['burst_type_' + activity]
        )

In [ ]:
X_train = X_train[[
    "id",
    "event_id",
    "window_30s",
    "burst_id",
    "burst_type",
    "burst_type_Nonproduction",
    "burst_type_Input",
    "burst_type_Remove/Cut",
    "burst_type_Replace",
    "burst_type_Paste",
    "is_new_burst_start",
    "is_new_burst_start_Nonproduction",
    "is_new_burst_start_Input",
    "is_new_burst_start_Remove/Cut",
    "is_new_burst_start_Replace",
    "is_new_burst_start_Paste",
    "burst_time_start",
    "burst_time_end",
    "burst_duration",
    "word_count_delta_burst",

    "down_time",
    "up_time",	
    "action_time",	
    "activity",	
    "activity_Nonproduction",
    "activity_Input",
    "activity_Remove/Cut",
    "activity_Replace",
    "activity_Paste",
    "down_event",	
    "up_event",	
    "text_change",
    "is_thought_delimiting_punctuation",
    "cursor_position",	
    "word_count",

    "cursor_position_vs_max",
    "cursor_position_cummax",
    "has_cursor_position_moved_right",

    "is_new_latest_string_start",
    "is_latest_string_end",
    "n_alphanum_char_added_to_latest_string",
    "rolling_length_latest_string",
    "length_latest_string",

    "word_count_lag1",
    "word_count_delta_event",

    "up_time_lag1",
    "latency_time",
    "preceding_pause_time",
    "preceding_pause_time_start_window",
    "preceding_pause_time_bucket",
    "rolling_pause_time",
    "rolling_pause_time_fraction",
    "total_pause",

    "burst_action_time_Nonproduction",
    "burst_action_time_Input",
    "burst_action_time_Remove/Cut",
    "burst_action_time_Replace",
    "burst_action_time_Paste"
    ]]


In [ ]:
vars_sum = (
    ['activity_' + x for x in ACTIVITY_CATEGORIES] 
    + ['is_new_burst_start'] 
    + ['is_new_burst_start_' + x for x in ACTIVITY_CATEGORIES]
    + ['word_count_delta_event']
    + ["is_thought_delimiting_punctuation"]
    + ["preceding_pause_time"]
    )

X_train_marginals_sum_wrt_time = (
    X_train
    .groupby('id')
    [vars_sum]
    .agg(sum)
    )
X_train_marginals_sum_wrt_time['delete_insert_ratio'] = (
    X_train_marginals_sum_wrt_time['activity_Remove/Cut'] / 
    X_train_marginals_sum_wrt_time['activity_Input'] 
    )

In [ ]:
X_train_marginals_central_tendency_wrt_time = (
    X_train
    .groupby('id')
    .agg(
        latency_time_p50 = ('latency_time', np.median),
        pause_time_p50 = ('preceding_pause_time', np.median),
        has_cursor_position_moved_right_mean = ('has_cursor_position_moved_right', 'mean'),
        burst_duration_mean = ('burst_duration', 'mean'),
        burst_duration_p50 = ('burst_duration', np.median),
        word_count_delta_burst_p50 = ('word_count_delta_burst', np.median),
        cursor_position_vs_max_avg = ('cursor_position_vs_max', 'mean'),
        length_latest_string_mean = ('length_latest_string', 'mean'),
        length_latest_string_stddev = ('length_latest_string', np.std)
        )
    )

In [ ]:
X_train_marginals_extremes_wrt_time = (
    X_train
    .groupby('id')
    .agg(
        pause_time_max=('preceding_pause_time', 'max'),
        initial_pause_time_max=('preceding_pause_time_start_window', 'max'),
        # approximation to, next longest pause after first long planning pause
        pause_time_p99=('preceding_pause_time', lambda x: x.quantile(0.99)),
        burst_duration_max=('burst_duration', 'max'),
        total_time=('up_time', 'max'),
        length_latest_string_max=('length_latest_string', 'max'),
        latency_time_min=('latency_time', 'min')
        )
    )

X_train_marginals_extremes_wrt_time['is_initial_pause_max_pause'] = (
    X_train_marginals_extremes_wrt_time['pause_time_max'] == 
    X_train_marginals_extremes_wrt_time['preceding_pause_time_start_window']
    ).astype(int)

In [ ]:
from scipy.stats import lognorm

pause_distr_summary_subjects = []

for X_train_subject in [x for _, x in X_train.groupby('id')]:

    shape, location, scale = lognorm.fit(X_train_subject['preceding_pause_time'].dropna())

    pause_distr_summary = pd.DataFrame({
        'pauses_lognorm_shape': [shape], 
        'pauses_lognorm_location': [location],
        'pauses_lognorm_scale': [scale]
        })
    pause_distr_summary.index = [X_train_subject['id'].iloc[0]]
    
    pause_distr_summary_subjects.append(pause_distr_summary)

X_train_marginals_distr_params_wrt_time = pd.concat(pause_distr_summary_subjects, axis=0)

In [ ]:
X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_sum_wrt_time, 
    X_train_marginals_central_tendency_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_wrt_time, 
    X_train_marginals_extremes_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_wrt_time, 
    X_train_marginals_distr_params_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

In [ ]:
for var in vars_sum:

    if var == 'preceding_pause_time':
        var_out = 'pause_time_fraction'
    else:
        var_out = var + '_per_s'

    X_train_marginals_wrt_time[var_out] = (
        (X_train_marginals_wrt_time[var] / X_train_marginals_wrt_time['total_time'])
        )
    
    if 'per_s' in var_out:
        X_train_marginals_wrt_time[var_out] *= 1000

X_train_marginals_wrt_time = (
    X_train_marginals_wrt_time
    .assign(
        keystroke_speed = lambda x: (x.activity_Input + x['activity_Remove/Cut']) / x.total_time,
        words_per_thought_delimiting_punctuation = lambda x: x.word_count_delta_event / x.is_thought_delimiting_punctuation,
        )
    )

In [ ]:
X_train_marginals_wrt_time.head()

In [ ]:
X_train_marginals_wrt_time.isnull().sum()

In [ ]:
X_train_by_window = (
    X_train
    .groupby(['id', 'window_30s'])
    [vars_sum + ['cursor_position_vs_max']]
    .agg(sum)
    .astype(float)
    .reset_index(drop=False)
    )

X_train_by_window['cursor_position_vs_max'] = (
    X_train_by_window['cursor_position_vs_max'] / 
    X_train_by_window[['activity_' + x for x in ACTIVITY_CATEGORIES]].sum(axis=1)
    )

X_train_by_window['delete_insert_ratio'] = (
    X_train_by_window['activity_Remove/Cut'] / 
    X_train_by_window['activity_Input'] 
    )

X_train_by_window['window_30s_idx'] = X_train_by_window.index

# for variability measure more comparable between writers,
# de-mean by writer. 
# Ex: higher-throughput writer incurs higher stddev, because values have higher abs value
time_rate_normalizers = [
    x
    for x in X_train_marginals_wrt_time.columns
    if 'per_s' in x 
    ]
# join method allows for merge on single index column
X_train_by_window = X_train_by_window.join(
    X_train_marginals_wrt_time[time_rate_normalizers],
    on='id',
    how='left'
)
for denom in time_rate_normalizers:
    level = denom.replace("_per_s", "")
    X_train_by_window[level] = (
        X_train_by_window[level] / 
        (X_train_by_window[denom].replace(0, None) * 30)
        )
    X_train_by_window[level] = X_train_by_window[level].fillna(1)
    
X_train_by_window = X_train_by_window.drop(columns=time_rate_normalizers)

X_train_by_window['preceding_pause_time'] = (
    X_train_by_window['preceding_pause_time'] / (1000 * 30)
    )

In [ ]:
X_train_windows_variation = (
    X_train_by_window
    .drop(columns=['window_30s', 'window_30s_idx'])
    .groupby(['id'])
    .agg(np.std)
    )

X_train_windows_variation.columns = [
    x + "_stddev"
    for x in X_train_windows_variation.columns
    ]

In [ ]:
X_train_windows_variation.head()

In [ ]:
X_train_transform = pd.merge(
    X_train_marginals_wrt_time,
    X_train_windows_variation,
    how='left',
    left_index=True,
    right_index=True
    )

In [ ]:
X_train_transform

In [ ]:
X_train_transform.isnull().mean()

In [ ]:
CURSOR_POSITION_VS_MAX_STDDEV_P50 = 246.6

X_train_transform['cursor_position_vs_max_stddev'] = (
    X_train_transform['cursor_position_vs_max_stddev'].fillna(CURSOR_POSITION_VS_MAX_STDDEV_P50)
)

In [ ]:
(
    X_train_transform
    .drop(columns='delete_insert_ratio_stddev')
    .to_pickle("./data/processed/X_train.pkl")
)